# DOF from specified object-space resolution in frontoparallel imaging 

In [1]:
from __future__ import print_function, division
import numpy as np
import iutils.optics.fourier as fou
import iutils.optics.goptics as go
import sympy as sy

In [2]:
sy.init_printing(use_latex='mathjax', print_builtin=False)

In [3]:
# symbols
# d : half depth of focus (diffraction based)
# lam : optical wavelength
# N : effective F-number
# v : image plane distance 
# u : object plane distance
# f : focal length
d, lam, N = sy.symbols('\delta_h, \lambda, N', real=True, positive=True)
u, v, f = sy.symbols('u, v, f', real=True)

The half depth of focus:

In [4]:
sy.Eq(d, 6.4*lam*N**2 / sy.pi)

                2        
           6.4⋅N ⋅\lambda
\delta_h = ──────────────
                 π       

We obtain the object space depth of field (DOF) extents by applying the Gaussian lens equation to the image distances $v + \delta$ (for the near extent) and $v - \delta$ (for the far extent):

In [5]:
DOF_n = (v + d)*f / (v + d - f)
DOF_f = (v - d)*f / (v - d - f)

In [6]:
DOF_n

f⋅(\delta_h + v)
────────────────
\delta_h - f + v

In [7]:
DOF_f

f⋅(-\delta_h + v)
─────────────────
-\delta_h - f + v

In [8]:
dof = DOF_f - DOF_n

In [9]:
dof

f⋅(-\delta_h + v)   f⋅(\delta_h + v)
───────────────── - ────────────────
-\delta_h - f + v   \delta_h - f + v

In [10]:
dof = dof.simplify().factor()

In [11]:
dof

                        2             
            2⋅\delta_h⋅f              
──────────────────────────────────────
(-\delta_h - f + v)⋅(\delta_h - f + v)

Now, 

In [12]:
mt = sy.symbols('m_t', real=True)
ro = sy.symbols('r_o', real=True, positive=True)

In [13]:
lam_equiv = mt / (1.22*ro*N)
lam_equiv

0.819672131147541⋅m_t
─────────────────────
         N⋅rₒ        

In [14]:
dh = 6.4*lam*N**2 / sy.pi
dh

     2        
6.4⋅N ⋅\lambda
──────────────
      π       

In [15]:
dh = dh.subs(lam, lam_equiv)
dh

5.24590163934426⋅N⋅m_t
──────────────────────
         π⋅rₒ         

In [16]:
dh = (5.25 * N * mt)  / (sy.pi * ro)
dh

5.25⋅N⋅m_t
──────────
   π⋅rₒ   

In [17]:
dof1 = dof.subs(d, dh)
dof1

                         2                      
                 10.5⋅N⋅f ⋅m_t                  
────────────────────────────────────────────────
     ⎛  5.25⋅N⋅m_t        ⎞ ⎛5.25⋅N⋅m_t        ⎞
π⋅rₒ⋅⎜- ────────── - f + v⎟⋅⎜────────── - f + v⎟
     ⎝     π⋅rₒ           ⎠ ⎝   π⋅rₒ           ⎠

In [18]:
dof1.simplify()

                             2                         
                  -10.5⋅π⋅N⋅f ⋅m_t⋅rₒ                  
───────────────────────────────────────────────────────
(5.25⋅N⋅m_t - π⋅rₒ⋅(f - v))⋅(5.25⋅N⋅m_t + π⋅rₒ⋅(f - v))

We will rewrite the above expression as:

$$
DOF(r_o) = \frac{10.5 \pi N f^2 m_t r_o}{\left[\pi r_o (v-f) - 5.25 N m_t\right] \left[\pi r_o (v-f) + 5.25 N m_t\right]}
$$

Substituting $(v-f)=\frac{v f}{u}$ and $m_t = \frac{v}{u}$, we can further simplify the above expression to obtain:

$$
DOF(r_o) = \frac{10.5 \pi N f^2 r_o}{m_t (\pi r_o f - 5.25 N) (\pi r_o f + 5.25 N)}
$$

### Verification with experimental observation

In [19]:
def dof_from_object_resolution(r, m, f, N):
    """returns the total diffraction based depth of field in the object space for 
    a prescribed object space resolution
    
    Parameters
    ----------
    r : real
        prescribed object space resolution (lp/mm)
    m : real
        absolute value of the transverse magnification between the
        conjugate planes
    f : real
        focal length (mm)
    N : real
        effective F-number
        
    Returns
    -------
    dof : real
        total depth of field (mm)
    """
    num = 10.5 * np.pi * N * f**2 * r
    a = np.pi*r*f
    b = 5.25*N
    deno = m * (a - b) * (a + b)
    return num/deno

def control_function(r, m, f, N, u):
    """
    """
    lamEqui = m/(1.22*N*r)
    dof, _, _, = fou.depth_of_field(f, N, u, lamEqui)
    return dof


In [20]:
# case 1
f = 180.0  # mm
N = 8.0
mt = 0.046
u = 4038.6 # mm
r = 2.0    # lp/mm

print('DOF (using control function) = {:2.4f} cm'.format(control_function(r, mt, f, N, u)/10))
print('DOF = {:2.4f} cm'.format(dof_from_object_resolution(r, mt, f, N)/10))

DOF (using control function) = 30.9837 cm
DOF = 29.1032 cm


In [26]:
# case 2
f = 180.0  # mm
N = 8.0
mt = 0.0545
u = 3437 # mm
#r = 3.937    # lp/mm
r = 3.94    # lp/mm

print('DOF (using control function) = {:2.4f} cm'.format(control_function(r, mt, f, N, u)/10))
print('DOF = {:2.4f} cm'.format(dof_from_object_resolution(r, mt, f, N)/10))

DOF (using control function) = 13.4800 cm
DOF = 12.4564 cm


DOF gain in case 1:

In [23]:
1219.2/ 291.032

4.189230050303747

DOF gain in case 2:

In [27]:
1219.2/124.456

9.7962332069165